## 1 import packages

In [1]:
#!/usr/bin/env python
# coding=utf-8
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
# sys.path.append("/search/odin/yangyuran/program/Anaconda3/envs/tensorflow/lib/python3.6/site-packages/")


import datetime
import os

import numpy as np
import math
import tensorflow as tf
import data_process
import model
import time
import matplotlib.image as mping
import pandas as pd

from sklearn.preprocessing import OneHotEncoder

/search/odin/yangyuran/program/Anaconda3/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/search/odin/yangyuran/program/Anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 2 指定参数

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

N_CLASSES = 2
IMG_W = 256  # resize the image, if the input image is too large, training will be very slow.
IMG_H = 256
IMG_C = 3
RATIO = 0.2  # take 20% of dataset as validation data
BATCH_SIZE = 64
CAPACITY = 100 + 3 * BATCH_SIZE
MAX_STEP = 3000  # with current parameters, it is suggested to use MAX_STEP>10k
learning_rate = 0.0001  # with current parameters, it is suggested to use learning rate<0.0001

train_dir = '/search/odin/xudongmei/working/datasets/bak2_crop/train_split/'
test_dir = '/search/odin/xudongmei/working/datasets/bak2_crop/test_split/'
logs_train_dir = './logs/train/batch_size=64/'
logs_val_dir = './logs/val/batch_size=64/'
LOGNAME = 'maps'

## 3. training()

In [3]:
def run_training():
    with tf.Graph().as_default():
        train, train_label, val, val_label = data_process.get_files(train_dir, RATIO)

        train_batch, train_label_batch = data_process.get_batch(train,
                                                                train_label,
                                                                IMG_W,
                                                                IMG_H,
                                                                BATCH_SIZE,
                                                                CAPACITY)
        val_batch, val_label_batch = data_process.get_batch(val,
                                                            val_label,
                                                            IMG_W,
                                                            IMG_H,
                                                            BATCH_SIZE,
                                                            CAPACITY)

        # 输入数据的命名空间
        with tf.name_scope("Input"):
            features = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMG_W, IMG_H, IMG_C], name="X_features")
            labels = tf.placeholder(tf.int32, shape=[BATCH_SIZE], name="Y_label")
            keep_prob = tf.placeholder(tf.float32, name='Keep_prob')
            one_hot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=N_CLASSES)

        # print(features.shape)
        # print(labels.shape)
        # print(one_hot_labels.shape)

        train_step, cross_entropy, logits, keep_prob = model.inference(features, one_hot_labels, BATCH_SIZE, N_CLASSES)

        accuracy = model.evaluation(logits, one_hot_labels)

        with tf.Session() as sess:
            saver = tf.train.Saver()

            sess.run(tf.global_variables_initializer())

            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)

            summary_op = tf.summary.merge_all()
            train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)
            val_writer = tf.summary.FileWriter(logs_val_dir)

            logger = data_process.train_log(LOGNAME)

            try:
                train_total_loss = 0.0
                train_total_acc = 0.0
                val_total_loss = 0.0
                val_total_acc = 0.0
                
                for step in np.arange(MAX_STEP):
                    if coord.should_stop():
                        break
                    start_time = time.time()
                    
                    tra_images, tra_labels = sess.run([train_batch, train_label_batch])
                    _, tra_loss, tra_acc = sess.run([train_step, cross_entropy, accuracy],
                                                 feed_dict={features: tra_images, labels: tra_labels, keep_prob: 0.5})
                    train_total_loss += tra_loss
                    train_total_acc += tra_acc
                    
                    val_images, val_labels = sess.run([val_batch, val_label_batch])
                    val_loss, val_acc = sess.run([cross_entropy, accuracy],
                                                 feed_dict={features: val_images, labels: val_labels,
                                                            keep_prob: 0.5})
                    val_total_loss += val_loss
                    val_total_acc += val_acc
                    
                    if step % 50 == 0:
                        duration = time.time() - start_time
                        logger.info("step %d: training accuracy %g, loss is %g (%0.3f sec)" % (
                            step, tra_acc, tra_loss, duration))

                        summary_str = sess.run(summary_op,
                                               feed_dict={features: tra_images, labels: tra_labels, keep_prob: 0.5})
                        train_writer.add_summary(summary_str, step)

                    if step % 100 == 0 or (step + 1) == MAX_STEP:
                        duration = time.time() - start_time
                        logger.info("step %d: validation accuracy %g, loss is %g (%0.3f sec)" % (
                            step, tra_acc, tra_loss, duration))

                        summary_str = sess.run(summary_op,
                                               feed_dict={features: val_images, labels: val_labels, keep_prob: 0.5})
                        val_writer.add_summary(summary_str, step)

                    if step % 1000 == 0 or (step + 1) == MAX_STEP:
                        checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
                        saver.save(sess, checkpoint_path, global_step=step)
                        
                print("Average loss in training set:", train_total_loss / MAX_STEP)
                print("Average acc in training set:", train_total_acc / MAX_STEP)
                print("Average loss in val set:", val_total_loss / MAX_STEP)
                print("Average loss in val set:", val_total_acc / MAX_STEP)
                      
            except tf.errors.OutOfRangeError:
                print('Done training -- epoch limit reached')
            finally:
                coord.request_stop()
            coord.join(threads)

In [ ]:
begin = datetime.datetime.now()
run_training()
end = datetime.datetime.now()
print("Training time is: ", end - begin)

Starting baidu2_crop images
Starting sogou2_crop images
# training samples: 3904, # validation samples: 976


Mon 28 May 2018 21:25:01 INFO <ipython-input-3-fd2e0f5d7f53> 59 75480 step 0: training accuracy 0.4375, loss is 7.11028 (1.680 sec)


Step 0, train loss = 7.11, train accuracy = 43.75%


Mon 28 May 2018 21:25:01 INFO <ipython-input-3-fd2e0f5d7f53> 76 75480 step 0: validation accuracy 0.4375, loss is 7.11028 (2.210 sec)


**  Step 0, val loss = 6.12, val accuracy = 42.19%  **


...

Step 2800, train loss = 6.09, train accuracy = 48.44%


Mon 28 May 2018 21:30:46 INFO <ipython-input-3-fd2e0f5d7f53> 76 75480 step 2800: validation accuracy 0.484375, loss is 6.08866 (0.363 sec)


**  Step 2800, val loss = 6.49, val accuracy = 50.00%  **


Mon 28 May 2018 21:30:53 INFO <ipython-input-3-fd2e0f5d7f53> 59 75480 step 2850: training accuracy 0.359375, loss is 7.91447 (0.120 sec)


Step 2850, train loss = 7.91, train accuracy = 35.94%


## 评估

In [ ]:
def evaluate():
    tf.reset_default_graph()
    batch_size = 1

    with tf.Graph().as_default():
        test_images, test_label = data_process.get_files(test_dir, 0, False)  # 获取测试图片的地址

        features = tf.placeholder("float32", shape=[batch_size, IMG_H, IMG_W, IMG_C],
                                  name="features")
        labels = tf.placeholder("int32", [batch_size], name="labels")
        one_hot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=N_CLASSES)

        train_step, cross_entropy, logits, keep_prob = model.inference(features, one_hot_labels, batch_size, N_CLASSES)
        values, indices = tf.nn.top_k(logits, 1)

        accuracy = model.evaluation(logits, labels)

        with tf.Session() as sess:
            saver = tf.train.Saver()
            ckpt = tf.train.get_checkpoint_state(logs_train_dir)
            if ckpt and ckpt.model_checkpoint_path:
                print('Restore the model from checkpoint %s' % ckpt.model_checkpoint_path)
                # Restores from checkpoint
                global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                saver.restore(sess, ckpt.model_checkpoint_path)
                print('Loading success, global_step is %s' % global_step)
            else:
                raise Exception('no checkpoint find')

            result = []
            total_acc = 0.0
            for i, test_image in enumerate(test_images):
                temp_dict = {}
                image = mping.imread(test_image)
                # print(image.shape)  # (468, 1316, 3)
                image = tf.image.resize_image_with_crop_or_pad(image, IMG_H, IMG_W)
                x = tf.image.per_image_standardization(image)
                x = sess.run(x)  # 将tensor转化为数组

                predictions = np.squeeze(
                    sess.run(indices, feed_dict={features: np.expand_dims(x, axis=0), keep_prob: 1}), axis=0)
                temp_dict['image_id'] = test_image
                temp_dict['label_id'] = predictions.tolist()[0]
                result.append(temp_dict)
#                 print('image %s is %d' % (test_image, predictions[0]))

                # 每次预测的准确率
                accuracies = sess.run(accuracy, feed_dict={features: np.expand_dims(x, axis=0),
                                                           labels: np.expand_dims(test_label[i], axis=0), keep_prob: 1.0})
                # 计算总的准确率
                total_acc += np.sum(accuracies)

            print("Final Testing Accurate: ", total_acc / len(test_images))

            df = pd.DataFrame(result)
            df.to_csv("result.csv")



In [ ]:
evaluate()